In [1]:
!pip install torch transformers gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.1/322.1 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.0/468.0 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.6 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.27.1
    Uninstalling huggingface-hub-0.27.1:
      Successfully uninstalled huggingface-hub-0.27.1


In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import gradio as gr

# ✅ Load Model from Hugging Face Hub
model_name = "Nensee/Llama-2-7b-chat-finetune"

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")

# Set padding token if necessary
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

...

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/195 [00:00<?, ?B/s]

In [3]:
from google.colab import drive

drive.mount('/content/drive')


Mounted at /content/drive


In [22]:
import os
import torch
import json
import re
import gradio as gr
from transformers import AutoModelForCausalLM, AutoTokenizer

# ✅ Model Setup (Uncomment when using the model)
# model_name = "Nensee/Llama-2-7b-chat-finetune"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")

# if tokenizer.pad_token is None:
#     tokenizer.pad_token = tokenizer.eos_token

# ✅ History File Path
HISTORY_FILE = "/content/drive/My Drive/chat_history.json"

# ✅ Load Chat History (Handles Empty Files)
def load_chat_history():
    if os.path.exists(HISTORY_FILE):
        try:
            with open(HISTORY_FILE, "r") as f:
                data = json.load(f)
                return data if isinstance(data, list) else []  # Ensure valid history
        except json.JSONDecodeError:  # Handle corrupted or empty file
            return []
    return []  # Return empty history if file doesn’t exist

# ✅ Save Chat History
def save_chat_history(history):
    with open(HISTORY_FILE, "w") as f:
        json.dump(history, f, indent=4)

# ✅ Clean Assistant Response
def clean_response(response):
    """Extract only complete sentences from the response."""
    response = response.split("User:")[0].strip()
    match = re.search(r"^(.*[.!?])\s?", response)
    return match.group(1).strip() if match else response

# ✅ Step 1: Update User Message Immediately
def update_user_message(user_input, chat_history):
    chat_history.append({"role": "user", "content": user_input})
    save_chat_history(chat_history)  # ✅ Save immediately after user message
    return chat_history, ""  # ✅ Clears input field immediately

# ✅ Step 2: Generate Bot Response Separately
def generate_response(chat_history):
    # ✅ Prepare conversation context
    formatted_messages = "\n".join([f"{msg['role'].capitalize()}: {msg['content']}" for msg in chat_history]) + "\nAssistant:"

    # ✅ Tokenize input
    inputs = tokenizer(formatted_messages, return_tensors="pt", truncation=True, max_length=1024).to("cuda")

    # ✅ Generate response
    model.eval()
    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_length=inputs['input_ids'].shape[1] + 50,
            temperature=0.7,
            do_sample=True,
            repetition_penalty=1.1,
            pad_token_id=tokenizer.eos_token_id
        )

    # ✅ Decode response
    decoded = tokenizer.decode(output[:, inputs["input_ids"].shape[-1]:][0], skip_special_tokens=True)

    # ✅ Clean response
    cleaned_response = clean_response(decoded)

    # ✅ Append bot response to history
    chat_history.append({"role": "assistant", "content": cleaned_response})

    # ✅ Save updated chat history
    save_chat_history(chat_history)

    return chat_history  # ✅ UI updates with bot response

# ✅ Gradio Chat UI with History
with gr.Blocks() as demo:
    chatbot = gr.Chatbot(type="messages")
    user_input = gr.Textbox(placeholder="Type your message here...")
    # clear_btn = gr.Button("Clear Chat")

    history_state = gr.State(load_chat_history())  # ✅ Load history on startup

    # def clear_history():
    #     save_chat_history([])  # ✅ Overwrite history with an empty list
    #     history_state.value = []  # ✅ Clear Gradio state
    #     return [], ""  # ✅ Return empty chat and clear input field

    def load_ui():
        return history_state.value  # ✅ Load chat history properly

    demo.load(load_ui, [], chatbot)

    # ✅ Step 1: Update UI Immediately with User Message
    user_input.submit(update_user_message, [user_input, history_state], [chatbot, user_input])\
              .then(generate_response, history_state, chatbot)  # ✅ Step 2: Generate Bot Response After

    # ✅ Ensure clear button resets input field too
    # clear_btn.click(clear_history, [], [chatbot, user_input])

# ✅ Launch Gradio App with Async Queue
demo.queue().launch(debug=True, share=True)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://759823521b7e745cf5.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://759823521b7e745cf5.gradio.live
